In [1]:
import os
from pathlib import Path
from flask import (
    Flask,
    request,
    abort
)
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import (
    Configuration, 
    ApiClient, 
    MessagingApi, 
    ReplyMessageRequest, 
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)
from line_token import *

config_file = Path("line_token.py")

app = Flask(__name__)
line_bot_api = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

@app.route('/')
def hello():
    return 'Hello, World!'

@app.route('/callback',methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        handler.handle(body, signature)
        return "OK"
    except InvalidSignatureError as e:
        print(e)
        abort(400)

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event: MessageEvent):
    with ApiClient(line_bot_api) as api_client:
        message = MessagingApi(api_client)
        reply_token = event.reply_token
        replyText = event.message.text
        print(f"client submit {replyText}")

        list_reply = [
            TextMessage(text=replyText)
        ]

        message.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=reply_token,
                messages=list_reply
            )
        )

In [2]:
port = int(os.environ.get('PORT', 5555))
app.run(host='0.0.0.0', port=port, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5555
 * Running on http://192.168.1.109:5555
Press CTRL+C to quit
127.0.0.1 - - [06/Aug/2024 09:59:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2024 09:59:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2024 09:59:31] "GET /favicon.ico HTTP/1.1" 404 -


client submit aaa


127.0.0.1 - - [06/Aug/2024 09:59:57] "POST /callback HTTP/1.1" 200 -
